In [70]:
# Algoritmo base
# https://www.cs.utexas.edu/~flame/Notes/NotesOnCholReal.pdf

# Alterações feitas:

# A matriz L = zero(A) está sendo criada fora da função
# As alterações estão sendo feitas na própria matriz A
# Utilizar .= e .-=, operações in-place, otimizaram a alocação

In [10]:
using(LinearAlgebra)

In [11]:
##### Nível 2

# Algoritmo do livro (applied numerical linear algebra)
function  Cholesky_factorize(A::Matrix{Float64})
    
    # L: matriz de zeros que será alterada, mesmo tamanho de A
    L = zero(A)
    
    # Número de linhas
    m = size(A, 1)

    # Loop principal
    for j in 1:m
        
        # Somatório de Ljk ao quadrado
        sumLjk = 0
        for k in 1:j-1
            sumLjk += L[j, k]^2
        end

        # Alterações na diagonal de L
        L[j, j] = (A[j, j] - sumLjk)^(0.5)

        # Segundo loop principal
        for i in j+1:m
            
            # Somatório do produto Lik e Ljk
            sumLikLjk = 0
            for k in 1:j-1
                sumLikLjk += L[i, k]*L[j, k]
            end

            # Alterações
            L[i, j] = (A[i, j] - sumLikLjk)/L[j, j]
        end
    end

    return L
end

Cholesky_factorize (generic function with 1 method)

In [12]:
# TESTANDO NÍVEL 2 - TEMPO E ALOCAÇÃO

n = 400
X = randn(n, n)

A = X*X'

@time Cholesky_factorize(A)
@time cholesky(A)

print("------------------------------------------------")

  0.026061 seconds (3 allocations: 1.221 MiB)
  0.004691 seconds (4 allocations: 1.221 MiB)
------------------------------------------------

In [13]:
### Nível 3 ######

function Cholesky_by_blocks(A::Matrix{Float64}, block_size::Int)
    # Número de linhas
    n = size(A)[1]
    for j in 1:block_size:n
        b = min(n - j + 1, block_size)
        # Realizando as alterações na própria matriz A, poupa alocações
        A[j:j+b-1, j:j+b-1] .= Cholesky_factorize(A[j:j+b-1, j:j+b-1])
        A[j+b:n, j:j+b-1] /= A[j:j+b-1, j:j+b-1]'
        A[j+b:n, j+b:n] .-= A[j+b:n, j:j+b-1]*A[j+b:n, j:j+b-1]'

    end
    return tril(A)
end

Cholesky_by_blocks (generic function with 1 method)

In [16]:
# TESTANDO NÍVEL 3 - ALOCAÇÃO E TEMPO - TESTE 1
n = 800
X = randn(n, n)
size_block = 512
A = X*X'

@time Cholesky_factorize(A)
@time cholesky(A)
@time Cholesky_by_blocks(A, size_block) # Essa deve ficar por último pois altera A
print("----------------------------------------------------------------------------------")

  0.374046 seconds (3 allocations: 4.883 MiB)
  0.038103 seconds (4 allocations: 4.883 MiB, 15.22% gc time)
  0.237205 seconds (49 allocations: 19.673 MiB, 2.22% gc time)
----------------------------------------------------------------------------------

In [695]:
# TESTANDO NÍVEL 3 - ALOCAÇÃO E TEMPO - TESTE 2
n = 3000
X = randn(n, n)
size_block = 2048
A = X*X'

@time Cholesky_factorize(A)
@time cholesky(A)
@time Cholesky_by_blocks(A, size_block) # Essa deve ficar por último pois altera A
print("----------------------------------------------------------------------------------")

 78.980518 seconds (3 allocations: 68.665 MiB)
  2.625674 seconds (4 allocations: 68.665 MiB)
 38.679976 seconds (49 allocations: 273.614 MiB, 8.69% gc time)
----------------------------------------------------------------------------------

In [696]:
# TESTANDO NÍVEL 3 - ALOCAÇÃO E TEMPO - TESTE 3
n = 5000
X = randn(n, n)
size_block = 2048
A = X*X'

@time Cholesky_factorize(A)
@time cholesky(A)
@time Cholesky_by_blocks(A, size_block) # Essa deve ficar por último pois altera A
print("----------------------------------------------------------------------------------")

550.065713 seconds (3 allocations: 190.735 MiB, 0.00% gc time)
 23.421346 seconds (4 allocations: 190.735 MiB, 46.87% gc time)
104.517186 seconds (79 allocations: 848.131 MiB, 21.14% gc time)
----------------------------------------------------------------------------------

In [18]:
# ERROS ! ! !
n = 500
X = randn(n, n)
size_block = 224

A = X*X'
B = copy(A)

L1 = cholesky(A).L
L2 = Cholesky_factorize(A)
L3 = Cholesky_by_blocks(A, size_block)


display(norm(B - L1*L1'))
display(norm(B - L2*L2'))
display(norm(B - L3*L3'))

3.59036593153747e-12

4.051936509396488e-12

2.269092769490994e-12

In [19]:
#### COMPARAÇÃO DOS RESULTADOS
X = randn(4, 4)
A =X*X'

display(Cholesky_factorize(A))
display(cholesky(A).L)
display(Cholesky_by_blocks(A, 2))

4×4 Matrix{Float64}:
  1.34659    0.0        0.0       0.0
 -0.900193   1.92343    0.0       0.0
 -1.19725   -1.5        1.92819   0.0
 -0.896621   0.782927  -0.414207  0.508874

4×4 LowerTriangular{Float64, Matrix{Float64}}:
  1.34659     ⋅          ⋅         ⋅ 
 -0.900193   1.92343     ⋅         ⋅ 
 -1.19725   -1.5        1.92819    ⋅ 
 -0.896621   0.782927  -0.414207  0.508874

4×4 Matrix{Float64}:
  1.34659    0.0        0.0       0.0
 -0.900193   1.92343    0.0       0.0
 -1.19725   -1.5        1.92819   0.0
 -0.896621   0.782927  -0.414207  0.508874

In [22]:
n = 10

a = randn(n, 1)
b = randn(n, 1)

function prod(a::Matrix{Float64}, b::Matrix{Float64})
        
    soma = 0
    l = size(a, 1)
        
    for i in 1:l
        soma += a[i]*b[i]
    end

    return soma
end

function prodzip(a::Matrix{Float64}, b::Matrix{Float64})
    
    soma = 0
    for (x, y) in zip(a, b)
        soma += x*y
    end

    return soma

end


@time sum(a.*b)
@time prod(a, b)
@time prodzip(a, b)


  0.000019 seconds (4 allocations: 208 bytes)
  0.000014 seconds
  0.000014 seconds


-0.2167463749787647

In [387]:
A = rand(5, 5)

function acessarA(A::Matrix{Float64})

    for x in A[1, 1:3]
        x
    end
end

@time A[1, 1:3]
@time acessarA(A)


  0.000701 seconds (2 allocations: 80 bytes)
  0.000015 seconds (1 allocation: 48 bytes)
